# Connect

In [1]:
# Panda hostname/IP and Desk login information of your robot
hostname = '172.16.0.2'
username = 'admin'
password = 'wanglab123'

# panda-py is chatty, activate information log level
import logging
logging.basicConfig(level=logging.INFO)

In [2]:
import panda_py

desk = panda_py.Desk(hostname, username, password, platform='fr3')
desk.unlock()
desk.activate_fci()

INFO:desk:Login succesful.
INFO:desk:Retaken control.


In [3]:
from panda_py import libfranka

panda = panda_py.Panda(hostname)
gripper = libfranka.Gripper(hostname)

INFO:panda:Connected to robot (172.16.0.2).


# Hand in eye calib

In [4]:
from panda_py import constants
from scipy.spatial.transform import Rotation as R
import pyrealsense2 as rs
import numpy as np
import matplotlib.pyplot as plt
import os

In [5]:
def enumerate_devices():
    """Enumerate connected RealSense cameras and return the first device serial number."""
    ctx = rs.context()
    devices = ctx.query_devices()
    if not devices:
        raise ValueError("No RealSense devices found.")
    return devices[0].get_info(rs.camera_info.serial_number)

def initialize_camera(serial_number):
    """Initialize the RealSense camera with the given serial number and return the pipeline and intrinsics."""
    pipeline = rs.pipeline()
    config = rs.config()
    config.enable_device(serial_number)
    config.enable_stream(rs.stream.color, 1280, 720, rs.format.rgb8, 30)
    config.enable_stream(rs.stream.depth, 1280, 720, rs.format.z16, 30)
    profile = pipeline.start(config)
    
    # Get intrinsics
    color_intrinsics = profile.get_stream(rs.stream.color).as_video_stream_profile().get_intrinsics()
    return pipeline, color_intrinsics

def save_intrinsics(intrinsics, path):
    """Save camera intrinsics to a file."""
    np.savez(path, fx=intrinsics.fx, fy=intrinsics.fy, ppx=intrinsics.ppx, ppy=intrinsics.ppy, coeffs=intrinsics.coeffs)

def capture_and_save_frames(pipeline, align, base_dir, frame_num):
    """Capture aligned color and depth frames from the camera and save them."""
    frames = pipeline.wait_for_frames()
    aligned_frames = align.process(frames)
    color_frame = aligned_frames.get_color_frame()
    depth_frame = aligned_frames.get_depth_frame()
    
    if not color_frame or not depth_frame:
        print("Failed to capture frames")
        return
    
    color_image = np.asanyarray(color_frame.get_data())
    depth_image = np.asanyarray(depth_frame.get_data())
    
    rgb_filename = f'{base_dir}/rgb/{frame_num}.png'
    depth_filename = f'{base_dir}/depth/{frame_num}.npy'
    
    plt.imsave(rgb_filename, color_image)
    np.save(depth_filename, depth_image)
    
    print(f"Saved {rgb_filename}")
    print(f"Saved {depth_filename}")

def generate_and_move_to_pose(init_pose, roll, pitch, yaw, z_add, x_add, y_add, max_roll_deviation, max_pitch_deviation, max_yaw_deviation):
    """Generate a new pose with turbulence and move the robot arm to it."""
    roll_turbulent = roll + np.random.uniform(-max_roll_deviation, max_roll_deviation)
    pitch_turbulent = pitch + np.random.uniform(-max_pitch_deviation, max_pitch_deviation)
    yaw_turbulent = yaw + np.random.uniform(-max_yaw_deviation, max_yaw_deviation)

    r = R.from_euler('xyz', [roll_turbulent, pitch_turbulent, yaw_turbulent], degrees=False)
    rotation_matrix = r.as_matrix()

    absolute_rotation_matrix = np.dot(init_pose[:3, :3], rotation_matrix)

    pose = init_pose.copy()
    pose[:3, :3] = absolute_rotation_matrix
    pose[2, 3] += z_add
    pose[0, 3] += x_add
    pose[1, 3] += y_add

    panda.move_to_pose(pose)
    
    return pose
    
def save_pose(pose, base_dir, frame_num):
    """Save the robot arm's pose to a file."""
    pose_filename = f'{base_dir}/poses/pose_{frame_num}.npy'
    np.save(pose_filename, pose)
    print(f"Saved pose to {pose_filename}")

In [6]:
def setup_directories(base_dir):
    os.makedirs(f'{base_dir}/rgb', exist_ok=True)
    os.makedirs(f'{base_dir}/depth', exist_ok=True)
    os.makedirs(f'{base_dir}/poses', exist_ok=True)

def initialize_pipeline():
    align = rs.align(rs.stream.color)
    serial_number = enumerate_devices()
    pipeline, color_intrinsics = initialize_camera(serial_number)
    return pipeline, align, color_intrinsics

def capture_images(start_frame, picture_nums, base_dir, init_pose, roll, pitch, yaw, z_add, x_add, y_add,
                   max_roll_deviation, max_pitch_deviation, max_yaw_deviation):
    setup_directories(base_dir)
    pipeline, align, color_intrinsics = initialize_pipeline()

    # Save the camera intrinsics
    save_intrinsics(color_intrinsics, f'{base_dir}/camera_intrinsics.npz')

    for frame_num in range(start_frame, start_frame + picture_nums):  # Capture images at 10 different poses
        pose = generate_and_move_to_pose(init_pose, roll, pitch, yaw, z_add, x_add, y_add,
                                         max_roll_deviation, max_pitch_deviation, max_yaw_deviation)
        capture_and_save_frames(pipeline, align, base_dir, frame_num)
        save_pose(pose, base_dir, frame_num)

    panda.move_to_start()

    pipeline.stop()


In [7]:
# Define a list of configurations
image_configs = [
    {
        'base_dir': './hand_in_eye',
        'init_pose': panda_py.fk(constants.JOINT_POSITION_START),
        'roll': 0.0,
        'pitch': 0.2,
        'yaw': 0.0,
        'z_add': 0.16,
        'x_add': 0.12,
        'y_add': 0.0,
        'max_roll_deviation': 0.05,
        'max_pitch_deviation': 0.05,
        'max_yaw_deviation': 0.05
    },
    {
        'base_dir': './hand_in_eye',
        'init_pose': panda_py.fk(constants.JOINT_POSITION_START),
        'roll': 0.3,
        'pitch': 0.35,
        'yaw': 0.0,
        'z_add': 0.1,
        'x_add': 0.0,
        'y_add': -0.25,
        'max_roll_deviation': 0.05,
        'max_pitch_deviation': 0.05,
        'max_yaw_deviation': 0.05
    },
    {
        'base_dir': './hand_in_eye',
        'init_pose': panda_py.fk(constants.JOINT_POSITION_START),
        'roll': -0.25,
        'pitch': 0.35,
        'yaw': 0.2,
        'z_add': 0.12,
        'x_add': 0.1,
        'y_add': 0.20,
        'max_roll_deviation': 0.05,
        'max_pitch_deviation': 0.05,
        'max_yaw_deviation': 0.05
    },
    {
        'base_dir': './hand_in_eye',
        'init_pose': panda_py.fk(constants.JOINT_POSITION_START),
        'roll': -0.25,
        'pitch': 0.2,
        'yaw': 0.2,
        'z_add': 0.20,
        'x_add': 0.11,
        'y_add': 0.1,
        'max_roll_deviation': 0.05,
        'max_pitch_deviation': 0.05,
        'max_yaw_deviation': 0.05
    }
]

# Iterate over the list of configurations and capture images
for i, config in enumerate(image_configs):
    capture_images(30*i, 30,config['base_dir'], config['init_pose'], config['roll'], config['pitch'], config['yaw'],
                   config['z_add'], config['x_add'], config['y_add'], config['max_roll_deviation'],
                   config['max_pitch_deviation'], config['max_yaw_deviation'])

INFO:panda:Initializing motion generation (moveToPose).
INFO:motion:Computed Cartesian trajectory: 1 waypoint, duration 0.14 seconds.
INFO:panda:Starting new controller (Trajectory).
INFO:panda:Stopping active controller (Trajectory).
INFO:panda:Initializing motion generation (moveToPose).
INFO:motion:Computed Cartesian trajectory: 1 waypoint, duration 0.23 seconds.
INFO:panda:Starting new controller (Trajectory).


Saved ./hand_in_eye/rgb/0.png
Saved ./hand_in_eye/depth/0.npy
Saved pose to ./hand_in_eye/poses/pose_0.npy


INFO:panda:Stopping active controller (Trajectory).
INFO:panda:Initializing motion generation (moveToPose).
INFO:motion:Computed Cartesian trajectory: 1 waypoint, duration 0.12 seconds.
INFO:panda:Starting new controller (Trajectory).


Saved ./hand_in_eye/rgb/1.png
Saved ./hand_in_eye/depth/1.npy
Saved pose to ./hand_in_eye/poses/pose_1.npy


INFO:panda:Stopping active controller (Trajectory).
INFO:panda:Initializing motion generation (moveToPose).
INFO:motion:Computed Cartesian trajectory: 1 waypoint, duration 0.28 seconds.
INFO:panda:Starting new controller (Trajectory).


...

Saved ./hand_in_eye/rgb/117.png
Saved ./hand_in_eye/depth/117.npy
Saved pose to ./hand_in_eye/poses/pose_117.npy


INFO:panda:Stopping active controller (Trajectory).
INFO:panda:Initializing motion generation (moveToPose).
INFO:motion:Computed Cartesian trajectory: 1 waypoint, duration 0.17 seconds.
INFO:panda:Starting new controller (Trajectory).


Saved ./hand_in_eye/rgb/118.png
Saved ./hand_in_eye/depth/118.npy
Saved pose to ./hand_in_eye/poses/pose_118.npy


INFO:panda:Stopping active controller (Trajectory).
INFO:panda:Initializing motion generation (moveToJointPosition).
INFO:motion:Computed joint trajectory: 1 waypoint, duration 2.72 seconds.
INFO:panda:Starting new controller (Trajectory).


Saved ./hand_in_eye/rgb/119.png
Saved ./hand_in_eye/depth/119.npy
Saved pose to ./hand_in_eye/poses/pose_119.npy
